In [1]:
import requests
import urllib
import pandas as pd
import os
import zipfile
import sqlalchemy as db
import matplotlib.pyplot as plt
import cufflinks as cf
import datetime

In [2]:
cf.set_config_file(offline=True)

# Get Stock Price

In [3]:
def get_all_stock():
    con = db.create_engine("sqlite:///C:/Users/zheji/Desktop/TradingTools/dailyDB.sqlite")
    dfs = []
    for table in con.table_names():
        tmp = pd.read_sql(table, con)
        tmp['ticker'] = table
        tmp['1dayret%'] = (tmp['Adj Close'] - tmp['Adj Close'].shift(1))/tmp['Adj Close'].shift(1) * 100
        tmp['nextdayret%'] = (tmp['Adj Close'].shift(-1) - tmp['Adj Close'])/ tmp['Adj Close'] * 100
        dfs.append(tmp)
    return pd.concat(dfs)
stock_df = get_all_stock()

# FINRA DARK POOL VIEW

In [4]:
# load dark pool and lit pool shorting data
con1 = db.create_engine('sqlite:///'+'shortDB'+'.sqlite')
ts = pd.read_sql_query('SELECT * from DARK', con1)
con2 = db.create_engine('sqlite:///'+'shortDB'+'.sqlite')
ts_lit = pd.read_sql_query('SELECT * from Lit', con2)
ts = ts.drop_duplicates()
ts_lit = ts_lit.drop_duplicates()

In [5]:
short = pd.merge(ts, ts_lit, left_on=['Date', 'Symbol'], right_on=['Date', 'Symbol'], how='inner')  # cboe + finra 
view = pd.merge(short, stock_df, left_on=['Date', 'Symbol'], right_on=['Date', 'ticker'], how='inner') # cboe + finra + stock

In [6]:
view['Lit_VolumeRatio'] = view['CBOE_TotalVolume']/view['Volume']
view['Dark_VolumeRatio'] = view['NMS_TotalVolume']/view['Volume']
view['Lit_Dark_Ratio'] = view['CBOE_TotalVolume']/view['NMS_TotalVolume']
view.head(5)

,index_x,Symbol,NASDAQCAR_ShortVolume,NASDAQCAR_TotalVolume,NYSE_ShortVolume,NYSE_TotalVolume,Date,NMS_ShortVolume,NMS_TotalVolume,NASDAQCAR_ShortRatio,...,Low,Close,Adj Close,Volume,ticker,1dayret%,nextdayret%,Lit_VolumeRatio,Dark_VolumeRatio,Lit_Dark_Ratio
0,0,A,89508.0,412346.0,36071.0,146290.0,2019-02-20,125579.0,558636.0,0.217070,...,77.330002,78.559998,77.697357,2076500.0,A,1.341609,-0.852852,0.125743,0.269028,0.467397
1,5,AAL,844109.0,1372781.0,319835.0,584804.0,2019-02-20,1163944.0,1957585.0,0.614890,...,34.770000,35.060001,34.596169,6969900.0,AAL,-1.155894,-0.228190,0.165140,0.280863,0.587973
2,9,AAP,272139.0,469307.0,18423.0,54741.0,2019-02-20,290562.0,524048.0,0.579874,...,162.589996,166.000000,165.279968,1990200.0,AAP,-0.682054,-2.825314,0.123362,0.263314,0.468497
3,10,AAPL,2626567.0,5520623.0,1587174.0,3256997.0,2019-02-20,4213741.0,8777620.0,0.475774,...,170.990005,172.029999,169.807388,26114400.0,AAPL,0.643545,-0.563850,0.121747,0.336122,0.362211
4,18,ABBV,435128.0,1587068.0,39094.0,260434.0,2019-02-20,474222.0,1847502.0,0.274171,...,79.139999,79.989998,73.424461,5564100.0,ABBV,-0.658219,-1.550194,0.118325,0.332040,0.356357


In [7]:
def single_stock_view(view, dix_view, stock):
    df = view[view['Symbol']==stock]
    if df.shape[0] != 0:
        display_cols = ['Date','Symbol', 'NMS_ShortRatio','CBOE_ShortRatio',  'Dark_VolumeRatio', 'Lit_VolumeRatio','1dayret%']
        df = df[display_cols]
        df.index = df.Date
        plot_cols = ['NMS_ShortRatio','CBOE_ShortRatio',  'Dark_VolumeRatio', 'Lit_VolumeRatio']
        df[plot_cols].iplot(kind='line')
    else:
        df2 = dix_view[dix_view['Symbol'] == stock]
        df2.index = df2.Date
        df2['Lit_Dark_Ratio'] = df2['CBOE_ShortVolume'] / df2['NMS_ShortVolume']
        plot_cols = ['NMS_ShortRatio', 'CBOE_ShortRatio', 'Lit_Dark_Ratio']
        df2[plot_cols].iplot(kind='line')
        
def single_stock_bound_view(view, stock, plot_cols, window_size, bound, recent_n):
    df = view[view['Symbol']==stock]
    df.index = df.Date
    for col in plot_cols:
        df[col+'_upper'] = df[col].rolling(window_size).mean() + bound * df[col].rolling(window_size).std()
        df[col+'_lower'] = df[col].rolling(window_size).mean() - bound * df[col].rolling(window_size).std()
        df[col+'_mean'] = df[col].rolling(10).mean() 
        df[[col, col+'_mean', col+'_upper', col+'_lower']][-1*recent_n:].iplot(kind='line')
    df['1dayret%'][-1*recent_n:].iplot(kind='bar')

def multi_stock_ts_view(view, stocks, show_col, recent_n):
    show = pd.DataFrame()
    for stock in stocks:
        df = view[view['Symbol']==stock]
        df.index = df.Date
        show[stock] = df[show_col]
    show[-1*recent_n:].iplot(kind='line')

In [8]:
multi_stock_ts_view(short, ['PDD', 'W', 'SHOP', 'QQQ', 'SPY', 'AAPL', 'UAL', 'SAVE', 'MGM', 'WFC', 'XLU', 'FLIR', 'FSLY'], 'NMS_ShortRatio', 30) # Dark

In [9]:
multi_stock_ts_view(short, ['PDD', 'W', 'SHOP', 'QQQ', 'SPY', 'AAPL', 'UAL', 'SAVE', 'MGM', 'WFC', 'XLU', 'FLIR'], 'CBOE_ShortRatio', 30)

In [15]:
ticker = 'DIS'
single_stock_view(view, short, ticker)

In [16]:
single_stock_bound_view(short, ticker, ['NMS_ShortRatio', 'CBOE_ShortRatio'], 50, 1, 90)

KeyError: '1dayret%'

# Screener View

In [18]:
min_vol = 500000
date = '2020-07-02'
pd.options.display.max_rows = 500
display_cols = ['Date','Symbol', 'NMS_ShortRatio','CBOE_ShortRatio',  'Dark_VolumeRatio', 'Lit_VolumeRatio','1dayret%']
scr_view = view[(view['NMS_TotalVolume'] > min_vol)&(view['Date']==date)]
scr_view.sort_values('NMS_ShortRatio', ascending= False)[display_cols]

,Date,Symbol,NMS_ShortRatio,CBOE_ShortRatio,Dark_VolumeRatio,Lit_VolumeRatio,1dayret%
144054,2020-07-02,AMCR,0.880588,0.918679,0.161506,0.125773,2.613751
144107,2020-07-02,CHRW,0.849855,0.795389,0.284066,0.147920,0.113799
144208,2020-07-02,GM,0.771918,0.385942,0.519280,0.084681,1.121798
144254,2020-07-02,IWM,0.732503,0.773862,0.225042,0.271579,0.408960
144232,2020-07-02,HST,0.705814,0.795157,0.216732,0.144806,0.550959
144434,2020-07-02,UA,0.704989,0.767823,0.358926,0.166230,0.346829
144406,2020-07-02,SPY,0.704381,0.574040,0.272639,0.257283,0.550696
144436,2020-07-02,UAL,0.684558,0.663788,0.642786,0.086341,-0.899859
144111,2020-07-02,CLX,0.657737,0.759780,0.326537,0.130038,1.179577
144278,2020-07-02,LEN,0.649977,0.693470,0.223707,0.135385,0.652391


In [16]:
scr_view.sort_values('1dayret%', ascending= False)[display_cols]

,Date,Symbol,NMS_ShortRatio,CBOE_ShortRatio,Dark_VolumeRatio,Lit_VolumeRatio,1dayret%
142040,2020-06-25,NOV,0.625087,0.648892,0.239926,0.195625,9.830209
141741,2020-06-25,ACN,0.470317,0.772850,0.341987,0.141713,7.669447
142030,2020-06-25,NBL,0.557117,0.611239,0.274052,0.154615,6.263498
142079,2020-06-25,PXD,0.705119,0.680904,0.345468,0.079901,6.011277
141868,2020-06-25,DXC,0.609242,0.574010,0.384909,0.105351,5.784574
141864,2020-06-25,DRI,0.464718,0.685083,0.430019,0.127931,5.338228
142138,2020-06-25,UAL,0.558397,0.577455,0.633967,0.085873,4.898697
141893,2020-06-25,FCX,0.484522,0.621138,0.298705,0.160815,4.824979
141921,2020-06-25,HAL,0.396891,0.511239,0.389938,0.179477,4.806076
142163,2020-06-25,WFC,0.355152,0.466139,0.383381,0.167111,4.785605


# Process Cboe Detailed Short Sales Data

In [ ]:
def load_stock_data(num_days = 9999):
    output = []
    for s in con.table_names():
        tmp = pd.read_sql(s, con)[-1*num_days:]
        tmp['Ticker'] = s
        output.append(tmp)
    stock_df = pd.concat(output)
    stock_df.index = stock_df['Ticker']
    del stock_df['Ticker']
    return stock_df

def analyze_short_volume(df, day_df):
    print(df.head(5))
    df['value'] = df['Size'] * df['Price']
    summary = pd.DataFrame()
    other = []
    groupdf = df.groupby('Symbol')
    summary['total_short_volume'] = groupdf['Size'].sum()
    summary['total_trades'] = groupdf['Time'].count()
    summary['average_price'] = groupdf['value'].sum()/summary['total_short_volume']
    summary['largest_block'] = groupdf['Size'].max()
    summary['median_size'] = groupdf['Size'].median()
    summary['max'] = df.iloc[groupdf['Size'].idxmax(), 1]
    
    summary = pd.merge(summary, day_df, left_index = True, right_on = 'Ticker')
    summary['day_mid_price'] = (summary['Open'] + summary['Close'] + summary['High'] + summary['Close'])/4
    print(summary.head(5))
    #summary = summary[summary.index.isin(con.table_names())]
    
    #load stock data
#     stock_df = load_stock_data(1)
#     summary = pd.concat([stock_df, summary], axis=1 )
#     summary = summary.dropna()
#     print(summary[['Open', 'High', 'Low', 'Close']].mean())
#     summary['stock_median_price'] = summary[['Open', 'High', 'Low', 'Close']].mean()
#     summary['short_vol_ratio'] = summary['total_short_volume']/summary['Volume'] * 100 
#     display_cols = ['Adj Close', 'total_short_volume', 'short_vol_ratio', 'stock_median_price', 'average_price']
#     return summary[display_cols]

def load_all_stock():
    dfs = []
    for s in con.table_names():
        tmp = pd.read_sql(s, con)
        tmp['Ticker'] = s
        dfs.append(tmp)
    return pd.concat(dfs)

In [ ]:
stock_df = load_all_stock()

In [ ]:
date='2020-04-20'
exchanges = ['EDGA', 'EDGX', 'BYX', 'BZX']
datestr = ''.join(date.split('-'))
con = db.create_engine('sqlite:///'+'yahooData'+'.sqlite')
day_stock = stock_df[stock_df['Date'] == date]
exch_dfs = []
for exch in exchanges:
    with open('DIX_data/'+exch+datestr+'.txt') as f:
        next(f)
        output = []
        for line in f:
            token = line.split('|')
            output.append({'Symbol': token[1],
                           'Time': token[3],
                           'Size': int(token[5]),
                           'Price': float(token[6]),
                           })
        df = pd.DataFrame(output)
        df['exchange'] = exch
        exch_dfs.append(df)

all_df = pd.concat(exch_dfs)

In [ ]:
all_df

In [ ]:
all_df['value'] = all_df['Size'] * all_df['Price']
summary = pd.DataFrame()
other = []
groupdf = all_df.groupby('Symbol')
summary['total_short_volume'] = groupdf['Size'].sum()
summary['total_trades'] = groupdf['Time'].count()
summary['average_price'] = groupdf['value'].sum()/summary['total_short_volume']
summary['largest_block'] = groupdf['Size'].max()
summary['median_size'] = groupdf['Size'].median()
summary = pd.merge(summary, day_stock, left_index = True, right_on = 'Ticker', how = 'inner')



In [ ]:
summary['short_ratio'] = summary['total_short_volume'] / summary['Volume']
summary['day_mid_price'] = (summary['Open'] + summary['Close'] + summary['High'] + summary['Close'])/4
summary.sort_values('short_ratio', ascending=False)

In [ ]:
def load_all_stock():
    dfs = []
    for s in con.table_names():
        dfs.append(pd.read_sql(s, con))
    return pd.concat(dfs)

def analyze_short_volume(df):
    df['value'] = df['Size'] * df['Price']
    summary = pd.DataFrame()
    other = []
    groupdf = df.groupby('Symbol')
    summary['total_short_volume'] = groupdf['Size'].sum()
    summary['total_trades'] = groupdf['Time'].count()
    summary['average_price'] = groupdf['value'].sum()/summary['total_short_volume']
    summary['largest_block'] = groupdf['Size'].max()
    summary['median_size'] = groupdf['Size'].median()
    summary['max'] = groupdf['Size'].idxmax()
    summary = summary[summary.index.isin(con.table_names())]
    
    #load stock data
    stock_df = load_stock_data(1)
    summary = pd.concat([stock_df, summary], axis=1 )
    summary = summary.dropna()
    print(summary[['Open', 'High', 'Low', 'Close']].mean())
    summary['stock_median_price'] = summary[['Open', 'High', 'Low', 'Close']].mean()
    summary['short_vol_ratio'] = summary['total_short_volume']/summary['Volume'] * 100 
    display_cols = ['Adj Close', 'total_short_volume', 'short_vol_ratio', 'stock_median_price', 'average_price']
    return summary[display_cols]